In [1]:
import time
from datetime import datetime
import pandas as pd
from dateutil import tz
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
import urllib.request
from bs4 import BeautifulSoup
import ssl
import os
import time
import base64
import requests
import instaloader
import urllib.request
from bs4 import BeautifulSoup
import ssl
import numpy as np

In [2]:


class InstagramBot:
    def __init__(self, email, password):
        # Path to your Chromedriver executable
        chromedriver_path = r"C:\Users\harsh\Downloads\chromedriver-win32\chromedriver-win32\chromedriver.exe"
        self.service = Service(chromedriver_path)
        self.browser = webdriver.Chrome(service=self.service)
        self.email = email
        self.password = password

    def signIn(self):
        self.browser.get('https://www.instagram.com/accounts/login/')
        print(self.browser.title)
        
        # Wait for the login fields to appear
        emailInput = WebDriverWait(self.browser, 10).until(
            EC.presence_of_element_located((By.NAME, 'username'))
        )
        passwordInput = WebDriverWait(self.browser, 10).until(
            EC.presence_of_element_located((By.NAME, 'password'))
        )

        emailInput.send_keys(self.email)
        passwordInput.send_keys(self.password)
        passwordInput.send_keys(Keys.ENTER)

        # Wait for the login process to complete
        time.sleep(5)
        
        # Handle popups (adjust selectors as necessary)
        try:
            okay = WebDriverWait(self.browser, 10).until(
                EC.presence_of_element_located((By.XPATH, "//button[contains(text(), 'Not Now')]"))
            )
            okay.click()
        except Exception as e:
            print("No 'Not Now' button found")

        try:
            notnow = WebDriverWait(self.browser, 10).until(
                EC.presence_of_element_located((By.XPATH, "//button[contains(text(), 'Not Now')]"))
            )
            notnow.click()
        except Exception as e:
            print("No 'Not Now' button found")

    def navigate_to_user(self, username):
        self.browser.get(f'https://www.instagram.com/{username}/')
        time.sleep(5)  # Wait for the page to load

    def scrape_posts(self, num_posts=5):
        post_links = []
        while len(post_links) < num_posts:
            links = self.browser.find_elements(By.XPATH, "//a[contains(@href, '/p/')]")
            for link in links:
                post_url = link.get_attribute('href')
                if post_url not in post_links:
                    post_links.append(post_url)
                if len(post_links) >= num_posts:
                    break
            self.browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2)  # Adjust sleep time as needed

        print(f"Scraped {len(post_links)} posts")
        return post_links[:num_posts]


    def save_image(self, url, filename):
        print(f"Saving image to {filename}")
        response = requests.get(url)
        with open(filename, 'wb') as file:
            file.write(response.content)

    def save_base64_image(self, data_url, filename):
        print(f"Saving base64 image to {filename}")
        header, encoded = data_url.split(",", 1)
        data = base64.b64decode(encoded)
        with open(filename, 'wb') as file:
            file.write(data)



In [3]:
def get_instagram_post_details(post_url):
    L = instaloader.Instaloader()
    # L.login('aniruthsince2003', 'Myntra2024#')
    try:
        post = instaloader.Post.from_shortcode(L.context, post_url.split('/')[-2])
        likes = post.likes
        comments = post.comments
        date_posted = post.date_utc.strftime('%Y-%m-%d %H:%M:%S')
        return likes, comments, date_posted
    except Exception as e:
        return f"An error occurred: {e}", None, None

In [4]:
def get_instagram_image_url(post_url):
    # Initialize Instaloader
    L = instaloader.Instaloader()
    
    # Get the shortcode from the post URL
    shortcode = post_url.split('/')[-2]
    
    # Download the post metadata
    post = instaloader.Post.from_shortcode(L.context, shortcode)
    
    # Get the image URL
    image_url = post.url
    return image_url

In [5]:
def get_followers(username):
    ctx = ssl.create_default_context()
    url = f'https://www.instagram.com/{username}/'

    try:
        html = urllib.request.urlopen(url, context=ctx).read()
        soup = BeautifulSoup(html, 'html.parser')
        data = soup.find_all('meta', attrs={'property': 'og:description'})
        
        if data:
            text = data[0].get('content').split()
            followers = text[0]
            return followers
        else:
            return None
    
    except Exception as e:
        print(f"Error fetching data for {username}: {str(e)}")
        return None

In [6]:
# Create lists to store data
profile_names = []
followers_counts = []
post_links = []
image_urls = []
likes_counts = []
comments_counts = []
dates_posted = []

# Example usage:
bot = InstagramBot('aniruthsince2003', 'Myntra2024#')
bot.signIn

profiles = [
    "komalpandeyofficial",
    "thatbohogirl",
    "stylemeupwithsakshi",
    "nagmamirajkar",
    "ishaborah",
    "aashnahegde",
    "aayushiivermaa",
    "juhigodambe",
    "barkhasingh0308",
    "thechiquefacto",
    "rxjveer",
    "mohak.narang",
    "ayush007",
    "ssunnychoppra",
    "sankett25",
    "vipul_juneja",
    "iakashchoudhary",
    "shauryasanadhya"
]
for profile in profiles:
    bot.navigate_to_user(profile)
    post_links = bot.scrape_posts(num_posts=2)
    # Get followers count
    followers_count = get_followers(profile)
    print(followers_count)
    for post_url in post_links:
        # Get post information
        likes,comments,date = get_instagram_post_details(post_url)
        # Append data to lists
        profile_names.append(profile)
        followers_counts.append(followers_count)
        post_links.append(post_url)
        # image_urls.append(get_instagram_image_url(post_url))
        likes_counts.append(likes)
        comments_counts.append(comments)
        dates_posted.append(date)

df = pd.DataFrame({
    'Profile Name': profile_names,
    'Followers Count': followers_counts,
    'Post Link': post_links,
    'Image URL': image_urls,
    'Likes': likes_counts,
    'Comments': comments_counts,
    'Date Posted': dates_posted
})

Scraped 2 posts
2M


In [ ]:

def calculate_raw_score(l, c, f, d, post_type='normal'):
    if post_type == 'normal':
        return (0.7 * l + 0.3 * c) * (1 + f / 1000) / (d + 1)
    elif post_type == 'reel':
        return (0.3 * l + 0.7 * c) * (1 + f / 1000) / (d + 1)
    else:
        raise ValueError("Invalid post_type. Should be 'normal' or 'reel'.")

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

In [ ]:
# Calculate raw scores for each row
df['Raw Score Normal'] = df.apply(lambda row: calculate_raw_score(row['Likes'], row['Comments'], row['Followers Count'], pd.to_datetime('now') - pd.to_datetime(row['Date Posted']), 'normal'), axis=1)

In [ ]:

import instaloader
def get_instagram_likes(post_url):
    L = instaloader.Instaloader()
    L.login('aniruthsince2003', 'Myntra2024')
    try:
        post = instaloader.Post.from_shortcode(L.context, post_url.split('/')[-2])
        return post.likes
    except Exception as e:
        return f"An error occurred: {e}"

# Example usage
url = 'https://www.instagram.com/p/C9VGtY1tZz8/'  # Replace with the actual post URL
likes,comments,date = get_instagram_post_details(url)
print(f"Number of likes: {likes,comments,date}")


BadCredentialsException: Login error: Wrong password.